In [ ]:
import pandas as pd
import numpy as np

data1 = pd.read_csv("071a9bf4-29dc-4a10-96f4-1b995c88c20b.csv") #interacted 1, vaja 0-id tekitada, DONE
data2 = pd.read_csv("433d73df-1cae-47ee-88b1-6f585eb32adc.csv") #weekly txs mainnet, nan -> 0, DONE
data3 = pd.read_csv("4cdb4b81-5b24-4ddc-be7c-4c4678145f51.csv") #DEX.TRADES_transactions txs, DONE
data4 = pd.read_csv("7bc89bae-41bb-4a82-8e02-c6f3934dc353.csv") #ENS, vaja ritta 0-id tekitada, DONE
data5 = pd.read_csv("b7f4f21d-0288-4482-af84-1f807d37b384.csv")

df = pd.concat([data1,data2,data3,data4,data5])

df[['NFT_class_bought','NFTclass_sold','DEX_vol_class']] = df[['NFT_class_bought','NFTclass_sold','DEX_vol_class']].fillna(value="0")
df.fillna(0,inplace=True)

df = df.drop(columns=['address'], axis = 1)
df.head()

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


In [ ]:
data_dum = pd.get_dummies(df)
train, test = train_test_split(data_dum, test_size=0.3)

X_train = train.drop(columns=['interacted'], axis=1) 
y_train = train["interacted"] 

X_test = test.drop(columns=['interacted'], axis=1)
y_test = test["interacted"]

#Resampling
X_adasyn, y_adasyn = ADASYN().fit_resample(X_train, y_train) 
X_smote, y_smote = SMOTE().fit_resample(X_train, y_train) 

#Prediction model
#try hyper-parameter tuning
rf_adasyn = RandomForestClassifier(n_estimators = 50,max_depth = 5,random_state = 3).fit(X_adasyn,y_adasyn) 
rf_smote = RandomForestClassifier(n_estimators = 50,max_depth = 5,random_state = 3).fit(X_smote,y_smote) 

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
print("Best parms are: grid_search.best_params_")

probs_adasyn = rf_adasyn.predict(X_test)
probs_smote = rf_smote.predict(X_test)


print("Accuracy score: "+str(accuracy_score(y_test,probs_adasyn))+", with confusion matrix: \n"
      +str(confusion_matrix(y_test,probs_adasyn)))
print("\nAccuracy score: "+str(accuracy_score(y_test,probs_smote))+", with confusion matrix: \n"
      +str(confusion_matrix(y_test,probs_smote)))

In [66]:
unique, counts = np.unique(df.NFT_class_bought, return_counts=True)
dict(zip(unique, counts))

{'0': 4101, '10K+': 70, '2.5K-10K': 88, '500-2.5K': 111, '>500': 351}

In [67]:
#try correlations
df_corr = data_dum.corr()
df_corr

,NFT_sum_bought,NFT_txs_bought,NFT_sum_sold,NFT_txs_sold,weekly_txs mainnet,DEX_vol_traded,DEX_transactions,ens,interacted,NFT_class_bought_0,...,NFTclass_sold_0,NFTclass_sold_10K+,NFTclass_sold_2.5K-10K,NFTclass_sold_500-2.5K,NFTclass_sold_>500,DEX_vol_class_0,DEX_vol_class_10K+,DEX_vol_class_2.5K-10K,DEX_vol_class_500-2.5K,DEX_vol_class_>500
NFT_sum_bought,1.000000,0.802551,0.868942,0.701053,-0.029294,-0.005020,-0.008963,-0.036181,-0.013942,-0.170039,...,-0.158154,0.323934,0.016936,0.001057,-0.010053,0.006630,-0.004419,-0.003855,-0.002358,-0.001925
NFT_txs_bought,0.802551,1.000000,0.899147,0.806878,-0.046514,-0.007971,-0.014232,-0.057451,-0.022138,-0.266495,...,-0.237838,0.398392,0.065605,0.024616,0.004205,0.010527,-0.007017,-0.006122,-0.003745,-0.003057
NFT_sum_sold,0.868942,0.899147,1.000000,0.804821,-0.027164,-0.004655,-0.008311,-0.033550,-0.012928,-0.148919,...,-0.154953,0.327468,0.016829,-0.004425,-0.013336,0.006148,-0.004098,-0.003575,-0.002187,-0.001785
NFT_txs_sold,0.701053,0.806878,0.804821,1.000000,-0.048952,-0.008389,-0.014978,-0.060461,-0.023298,-0.251288,...,-0.278910,0.457622,0.084412,0.032600,0.003720,0.011079,-0.007385,-0.006443,-0.003941,-0.003217
weekly_txs mainnet,-0.029294,-0.046514,-0.027164,-0.048952,1.000000,-0.033643,-0.060067,-0.242469,-0.093434,0.172276,...,0.175302,-0.075145,-0.073070,-0.078590,-0.102089,0.044430,-0.029617,-0.025838,-0.015806,-0.012902
DEX_vol_traded,-0.005020,-0.007971,-0.004655,-0.008389,-0.033643,1.000000,0.583581,-0.041554,-0.016012,0.029524,...,0.030043,-0.012878,-0.012523,-0.013468,-0.017496,-0.757220,0.891981,0.248531,0.036546,0.002487
DEX_transactions,-0.008963,-0.014232,-0.008311,-0.014978,-0.060067,0.583581,1.000000,-0.074190,-0.028589,0.052713,...,0.053639,-0.022993,-0.022358,-0.024047,-0.031237,-0.731827,0.512161,0.403415,0.261221,0.200062
ens,-0.036181,-0.057451,-0.033550,-0.060461,-0.242469,-0.041554,-0.074190,1.000000,-0.115401,0.212781,...,0.216519,-0.092813,-0.090250,-0.097067,-0.126092,0.054876,-0.036580,-0.031913,-0.019522,-0.015936
interacted,-0.013942,-0.022138,-0.012928,-0.023298,-0.093434,-0.016012,-0.028589,-0.115401,1.000000,0.081994,...,0.083434,-0.035765,-0.034777,-0.037404,-0.048588,0.021146,-0.014096,-0.012297,-0.007523,-0.006141
NFT_class_bought_0,-0.170039,-0.266495,-0.148919,-0.251288,0.172276,0.029524,0.052713,0.212781,0.081994,1.000000,...,0.665595,-0.390543,-0.291336,-0.266546,-0.323157,-0.038990,0.025990,0.022674,0.013870,0.011323


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4721 entries, 0 to 200
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NFT_sum_bought      4721 non-null   float64
 1   NFT_class_bought    4721 non-null   object 
 2   NFT_txs_bought      4721 non-null   float64
 3   NFT_sum_sold        4721 non-null   float64
 4   NFTclass_sold       4721 non-null   object 
 5   NFT_txs_sold        4721 non-null   float64
 6   weekly_txs mainnet  4721 non-null   float64
 7   DEX_vol_traded      4721 non-null   float64
 8   DEX_vol_class       4721 non-null   object 
 9   DEX_transactions    4721 non-null   float64
 10  ens                 4721 non-null   float64
 11  interacted          4721 non-null   float64
dtypes: float64(9), object(3)
memory usage: 479.5+ KB
